In [124]:
import os
import sys
import glob

from subprocess import Popen, PIPE, STDOUT, call

In [2]:
from set_test import list_test

In [3]:
files = sorted(glob.glob('./tasks/*'))
files

['./tasks/__pycache__',
 './tasks/task1.py',
 './tasks/task2.py',
 './tasks/task3.py']

In [3]:
os.system('python ./tasks/task1.py')

256

In [43]:
py = 'python'
file = './tasks/task1.py'

p = Popen('python ./tasks/task1.py',
          stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
grep_stdout = p.communicate(input=b'3\n3\n3\n', timeout=6*60)[0]
#type(grep_stdout.decode('866', errors='ignore')) # Для винды 10 кодировка по умолчанию 866
grep_stdout.decode()

TimeoutExpired: Command 'python ./tasks/task1.py' timed out after 360 seconds

In [5]:
#grep_stdout.decode('866', errors='ignore')
grep_stdout.decode()

'9\n'

In [34]:
p = Popen('python -m pycodestyle --ignore=E265,W292 ./tasks/task1.py', stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
grep_stdout = p.communicate()[0]
#type(grep_stdout.decode('866', errors='ignore')) # Для винды 10 кодировка по умолчанию 866
grep_stdout.decode()

''

In [35]:
def pep8_checker(filename):
    """
    Check codestyle by pycodestyle
    """
    p = Popen(f'python -m pycodestyle --ignore=E265,W292 {filename}',
              stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
    grep_stdout = p.communicate()[0]
    #type(grep_stdout.decode('866', errors='ignore')) # Для винды 10 кодировка по умолчанию 866
    pystyle_message = grep_stdout.decode()
    if pystyle_message:
        return 'NOPEP8', pystyle_message
    else:
        return 'OK', 'Your codestyle is PEP8'

In [36]:
pep8_checker('./tasks/task1.py')

('OK', 'Your codestyle is PEP8')

In [25]:
def result_checker(filename, input_data, output_data):
    """
    Check results of programs
    """
    p = Popen(f'python {filename}', stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
    grep_stdout = p.communicate(input=input_data, timeout=6*60)[0]
    if grep_stdout.decode() == output_data:
        return "Test is passed"
    else:
        return "Test isn't passed", grep_stdout.decode()

In [33]:
test_input = [b'3\n3\n3\n', b'1\n1\n-2\n', b'3\n3\n3\n', b'3\n3\n3\n', b'3\n3\n3\n']
test_output = ['9\n', '0\n', '9\n','9\n','9\n']

for tst_in, tst_out in zip(test_input, test_output):
    print(result_checker('./tasks/task1.py', tst_in, tst_out))

Test is passed
Test is passed
Test is passed
Test is passed
Test is passed


## Создем код, способный проверять функции в модулях

In [4]:
import importlib.util

In [102]:
default_path = (
    f'/home/andr/Yandex.Disk/3.Programing/'
    f'balovstvo&checking/test_interesting_code/checker_task'
)
task_path = './tasks/'
os.chdir(default_path)

In [63]:
os.chdir(task_path)
spec = importlib.import_module('task2', 'task2.py')
importlib.reload(spec)
func_name = 'my_range'
input_part = '(5, 7, 8)'
exec(f'result = spec.{func_name}{input_part}')
del spec
os.chdir(default_path)

In [64]:
result

range(5, 7, 8)

In [66]:
exec(f'result = spec.{func_name}{input_part}')

NameError: name 'spec' is not defined

In [9]:
def retr_func(a, b): return a + b
a = 5
b = 6
inp = (a, b)
result = None
test_str = '"a" , "6"'
string_code = (f'result = {retr_func(*inp)}')
exec(string_code)

In [10]:
result

11

In [152]:
def result_checker_function(filename, name_function, input_data, output_data):
    try:
        spec = importlib.import_module(filename[:-3], filename)
        importlib.reload(spec)
    except:
        return "Error", f'Error in your file {filename}: {sys.exc_info()}'

    try:
        exec(f"global result; result = spec.{name_function}{input_data}")
    except:
        return "Error", f'Error in your file {filename}: {sys.exc_info()}'
    
    if result == output_data:
        return "Test is passed", "It is correct."
    else:
        return "Test isn't passed", result
    del spec
    try:
        del sys.modules[filename[:-3]]
    except KeyError:
        pass
    
    return None

In [155]:
os.chdir(default_path)
for test in list_test:
    if test[0] == 'codeinout':
        print('Run check input and output')
    elif test[0] == 'function':
        for idx_test, item_test in enumerate(test[-1]):
            os.chdir(task_path)
            msg, out = result_checker_function(
                'task2.py', 
                test[1], 
                item_test[0], 
                item_test[1])
            
            if msg == "Test is passed":
                print("Test is passed")
            elif msg == 'Error':
                print(out)
            elif msg == "Test isn't passed":
                print(item_test[0], item_test[1], out)
            else:
                print('Unexpected condition!!!')
            os.chdir(default_path)

(3, 6, 2) [3, 5] (3, 6, 2)
(0, 10, 3) [0, 3, 6, 9] (0, 10, 3)
(10, 0, 3) [] (10, 0, 3)
(10, 0, -3) [10, 7, 4, 1] (10, 0, -3)
(-5, 10, 3) [-5, -2, 1, 4, 7] (-5, 10, 3)
Run check input and output


In [151]:
test

('codeinout',
 [(b'Application', b'pplctn\n'),
  (b'StRtPn', b'StRtPn\n'),
  (b'AoEiGGYeUa', b'GG\n'),
  (b'AoEiAUEYeUa', b'\n')])